# LangChain Workshop 1: From Manual API Calls to LangChain Magic ✨

Welcome to your first LangChain adventure! In this notebook, we'll:
1. See how tedious it is to make raw API calls to language models
2. Discover how LangChain makes everything easier and more elegant
3. Learn about prompt templates and LLM chains
4. Build something fun along the way!

Let's dive in! 🚀

## Setup: Loading Our API Keys Securely 🔐

First, let's load our environment variables. 

In real life, **NEVER** put API keys directly in your code! For this Kaggle workshop only, you can set your API key in the `KAGGLE_BACKUP` variable below. Though locally, you should use a `.env` file with the following content: 
```.env
OPENAI_API_KEY="your-key"
```

In [ ]:
# Only if running this on Kaggle
KAGGLE_BACKUP = "sk-..."  # Replace with your actual key for Kaggle only

In [ ]:
# Check that libraries exist

try:
    import requests
    import langchain
    import langchain_openai
    from dotenv import load_dotenv
except ImportError as e:
    !pip install requests python-dotenv langchain-openai langchain
    import requests
    import langchain
    import langchain_openai
    from dotenv import load_dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify our API key is loaded (we'll only show the first few characters for security)
api_key = os.getenv("OPENAI_API_KEY", KAGGLE_BACKUP)
if api_key:
    print(f"✅ API key loaded successfully: {api_key[:12]}...")
else:
    print("❌ No API key found. Make sure you have a .env file with OPENAI_API_KEY=")
    print("   Example .env file contents:")
    print("   OPENAI_API_KEY=sk-your-key-here")

## Part 1: The Old Way - Manual API Calls 😤

Let's see what it takes to manually call the OpenAI API. Spoiler alert: it's not fun.

In [ ]:
import requests

def manual_openai_call(prompt, temperature=0.7, max_tokens=150):
    """
    Make a manual API call to OpenAI - the hard way!
    Look at all this boilerplate code we have to write...
    """
    
    # Set up headers
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Create the payload
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    try:
        # Make the API call
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=payload
        )
        
        # Check if the request was successful
        response.raise_for_status()
        
        # Parse the response
        result = response.json()
        return result["choices"][0]["message"]["content"]
        
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"
    except KeyError as e:
        return f"Error parsing response: {str(e)}"

# Test our manual function
prompt = "Write a haiku about debugging code"
response = manual_openai_call(prompt)
print("Manual API call result:")
print(response)

### 😱 Look at all that code!

That was a lot of work just to ask a simple question! We had to:
- Set up headers manually
- Create the correct payload structure
- Handle HTTP requests
- Parse JSON responses
- Handle errors

And this is just for one simple call. Imagine if we wanted to:
- Chain multiple AI operations together
- Use different models
- Add memory to conversations
- Connect to different data sources

The complexity would explode! 💥

## Part 2: The LangChain Way - Simple and Elegant ✨

Now let's see how LangChain makes this SO much easier!

In [ ]:
from langchain_openai import ChatOpenAI

# Create an LLM instance - that's it!
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=150,
    api_key=api_key
)

# Make the same call - but so much simpler!
response = llm.invoke("Write a haiku about debugging code")
print("LangChain result:")
print(response.content)

### 🎉 Much Better!

Look how much simpler that was! LangChain handled all the:
- API authentication
- Request formatting
- Response parsing
- Error handling

We can focus on what matters: **building cool AI applications!**

## Part 3: Prompt Templates - Making Prompts Reusable 📝

What if we want to ask similar questions with different inputs? Like different user queries?That's where prompt templates shine!

In [ ]:
from langchain.prompts import PromptTemplate

# Create a template for generating creative writing
creative_template = PromptTemplate(
    input_variables=["genre", "character", "setting"],
    template="""
    Write a short {genre} story (2-3 sentences) featuring:
    - Character: {character}
    - Setting: {setting}
    
    Make it creative and engaging!
    """
)

# TODO: Fill in the blanks to generate multiple stories
scenarios = [
    {"genre": "sci-fi", "character": "a robot barista", "setting": "Mars colony"},
    {"genre": "____", "character": "____", "setting": "____"},
    {"genre": "____", "character": "____", "setting": "____"}
]

print("🎭 Generated Stories:")
print("=" * 50)

for i, scenario in enumerate(scenarios, 1):
    # Format the prompt with our variables
    formatted_prompt = creative_template.format(**scenario)
    
    # Get the AI's response
    response = llm.invoke(formatted_prompt)
    
    print(f"Story #{i} ({scenario['genre'].title()}):")
    print(response.content)
    print("-" * 30)

### 🎯 Why Prompt Templates Rock:

1. **Reusability**: Write once, use many times with different inputs
2. **Consistency**: Same format every time
3. **Maintainability**: Easy to update prompts in one place
4. **Flexibility**: Mix and match variables easily

## Part 4: LLM Chains - Connecting Operations Together 🔗

Now for the real magic: chaining operations together! Let's build a "Story Critic" that first generates a story, then critiques it.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Template for generating stories
story_template = PromptTemplate(
    input_variables=["topic"],
    template="Write a very short story (3-4 sentences) about {topic}. Make it interesting!"
)

# Template for critiquing stories
critic_template = PromptTemplate(
    input_variables=["story"],
    template="""
    As a literary critic, provide a brief review of this story:
    
    "{story}"
    
    Give it a rating out of 5 stars and explain why. Be constructive but honest!
    """
)

# Create chains
story_chain = LLMChain(llm=llm, prompt=story_template)
critic_chain = LLMChain(llm=llm, prompt=critic_template)

def story_and_critique(topic):
    """Generate a story and then critique it - a two-step AI process!"""
    
    print(f"📚 Generating story about: {topic}")
    story = story_chain.run(topic=topic)
    print("\nGenerated Story:")
    print(story)
    
    print("\n" + "=" * 40)
    print("🎭 Now getting a critique...")
    critique = critic_chain.run(story=story)
    print("\nCritique:")
    print(critique)
    
    return story, critique

# Try our story critic!
story_and_critique("a programmer who discovers their code has become sentient")

## 🎯 Exercise 1: Build Your Own Chain

Your turn! Create a "Product Idea Generator and Validator" that:
1. Takes a market/industry as input
2. Generates a creative product idea for that market
3. Then analyzes the pros and cons of that product idea

Fill in the code below:

In [ ]:
# TODO: Create your templates and chains here!

# Template for generating product ideas
product_idea_template = PromptTemplate(
    input_variables=["market"],
    template="""
    # YOUR CODE HERE: Create a template that generates creative product ideas
    # for a given market/industry. Bonus points for funny ideas
    """
)

# Template for analyzing product ideas
analysis_template = PromptTemplate(
    input_variables=["product_idea"],
    template="""
    # YOUR CODE HERE: Create a template that analyzes a product idea
    # Include pros, cons, and feasibility assessment
    """
)

# TODO: Create your chains and test function

def generate_and_analyze_product(market):
    """Generate a product idea and then analyze it"""
    # YOUR CODE HERE: Implement the two-step process
    pass

# Test your function!
# generate_and_analyze_product("pet care")

### 💡 Solution (Don't peek until you try!)

In [ ]:
# Solution to Exercise 1

product_idea_template = PromptTemplate(
    input_variables=["market"],
    template="""
    Generate a creative and innovative product idea for the {market} market.
    Think outside the box! The product should solve a real problem but in a fun or unexpected way.
    Describe the product in 2-3 sentences.
    """
)

analysis_template = PromptTemplate(
    input_variables=["product_idea"],
    template="""
    Analyze this product idea as a business consultant:
    
    "{product_idea}"
    
    Provide:
    1. Top 3 pros/advantages
    2. Top 3 cons/challenges  
    3. Feasibility score (1-10) with brief explanation
    4. One recommendation for improvement
    """
)

product_chain = LLMChain(llm=llm, prompt=product_idea_template)
analysis_chain = LLMChain(llm=llm, prompt=analysis_template)

def generate_and_analyze_product(market):
    print(f"💡 Generating product idea for: {market}")
    product_idea = product_chain.run(market=market)
    print("\nProduct Idea:")
    print(product_idea)
    
    print("\n" + "=" * 50)
    print("📊 Analyzing the idea...")
    analysis = analysis_chain.run(product_idea=product_idea)
    print("\nAnalysis:")
    print(analysis)
    
    return product_idea, analysis

# Test the solution
generate_and_analyze_product("YOUR PRODUCT IDEA HERE")

## 🎯 Exercise 2: The Emoji Translator

Let's build something fun! Create an "Emoji Translator" that:
1. Takes normal text as input
2. Converts it to an emoji-heavy version
3. Then translates it back to normal English
4. Compares how close the round-trip translation is to the original

This will be a 3-step chain!

In [ ]:
# TODO: Build your 3-step Emoji Translator!

# Step 1: Convert text to emojis
to_emoji_template = PromptTemplate(
    input_variables=["text"],
    template="""
    # YOUR CODE HERE: Create a template that converts normal text to emoji-heavy text
    # Be creative with emoji usage!
    """
)

# Step 2: Convert emojis back to text
from_emoji_template = PromptTemplate(
    input_variables=["emoji_text"],
    template="""
    # YOUR CODE HERE: Create a template that interprets emoji text back to normal English
    """
)

# TODO: Create your chains and main function

def emoji_round_trip(text):
    """Take text on a round trip through emoji land!"""
    # Step 1: Convert text to emojis
    
    # Step 2: Convert emojis back to text
        
    # Step 3: Print the results

# Test cases
test_sentences = [
    "I love programming and drinking coffee while debugging code",
    "The weather is sunny and I want to go to the beach",
    "My cat is sleeping on my laptop keyboard again"
]

# Uncomment to test:
# for sentence in test_sentences:
#     emoji_round_trip(sentence)

### 💡 Solution (Try it yourself first!)

In [ ]:
# Solution to Exercise 2

to_emoji_template = PromptTemplate(
    input_variables=["text"],
    template="""
    Convert this text to a version that uses lots of emojis while keeping the core meaning:
    
    "{text}"
    
    Use emojis creatively to represent words, concepts, and emotions. Have fun with it!
    """
)

from_emoji_template = PromptTemplate(
    input_variables=["emoji_text"],
    template="""
    Interpret this emoji-heavy text back into normal English:
    
    "{emoji_text}"
    
    Try to understand what the emojis represent and write it as a clear English sentence.
    """
)

# Create chains
to_emoji_chain = LLMChain(llm=llm, prompt=to_emoji_template)
from_emoji_chain = LLMChain(llm=llm, prompt=from_emoji_template)

def emoji_round_trip(text):
    print(f"🔄 Starting with: {text}")
    
    # Step 1: Convert to emoji
    print("\n📝 Step 1: Converting to emojis...")
    emoji_version = to_emoji_chain.run(text=text)
    print(f"Emoji version: {emoji_version}")
    
    # Step 2: Convert back to text
    print("\n🔄 Step 2: Converting back to text...")
    back_to_text = from_emoji_chain.run(emoji_text=emoji_version)
    print(f"Back to text: {back_to_text}")
    
    return emoji_version, back_to_text

# Test it out!
test_sentences = [
    "I love programming and drinking coffee while debugging code",
    "The weather is sunny and I want to go to the beach"
]

for sentence in test_sentences:
    emoji_round_trip(sentence)
    print("\n" + "="*60 + "\n")

## 🎉 What We've Learned

Congratulations! You've just learned the foundations of LangChain:

### ✅ Key Concepts Covered:
1. **Manual API Calls vs LangChain**: Saw how much boilerplate code LangChain eliminates
2. **LLM Wrappers**: Simple, consistent interface to different language models
3. **Prompt Templates**: Reusable, parameterized prompts for consistency
4. **LLM Chains**: Connecting multiple AI operations in sequence

### 🚀 What's Next:
- **Notebook 2**: Data loaders from cool sources (YouTube, news, etc.) and output parsers
- **Notebook 3**: Advanced features like chat models, agents, and vectorstores
- **Exercise 4**: Building a full Streamlit chatbot!

### 💡 Pro Tips:
- Always use environment variables for API keys
- Start simple with templates, then build complexity
- Chain operations can be as creative as you want
- Test each step of your chain individually for debugging